# Exploration

In [ ]:
import robosuite as suite
import numpy as np

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
from torch.autograd import Variable

In [ ]:
class Args:
    hidden_size = 256
    max_mem_size=2000
    tau=0.001
    batch_size=64
    lr=0.001
    lr_actor = 0.0001
    lr_critic = 0.001
    epsilon=10000
    warmup=100
    gamma=0.99
    theta=0
    num_episodes=50
    horizon=10
    num_epochs=100
    env_name='Lift'
    robot='Panda'
    seed=1

args=Args()

env = suite.make(
        env_name=args.env_name,
        robots=args.robot,
        has_renderer=True,
        has_offscreen_renderer=False,
        use_camera_obs=False,
        use_object_obs=True,                    
        horizon = 150, 
        reward_shaping=True                 
    )
obs = env.reset()
state_dim = obs['robot0_robot-state'].shape[0]+obs['object-state'].shape[0]
agent = DDPG(state_dim, env.action_dim, args)
agent.actor = torch.load('/Users/peterfagan/Desktop/test-run.pt')
agent.actor.eval()
done=False

while done==False: 
    state = np.append(obs['robot0_robot-state'],obs['object-state'])
    action = agent.select_action(state)
    obs, reward,done, info = env.step(action)
    agent.observe(reward, state, done)
    env.render()   
